In [1]:
import os
import sqlite3
import requests
import json

import matplotlib.pyplot as plt
#import nfts.dataset
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import zeta

In [4]:
# Read the trading data json file
original_df = pd.read_json("Data/txn_history-2021-10-07.jsonl", lines=True)
original_df.sample(10)

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories
101482,Bid,0x5ff76f,,2021-08-23,2021-08-23,larvalabs,59.99,6448,None,None,[Female],"[Knitted Cap, Eye Patch]"
8866,Bid,0x717403,,2017-06-26,2017-06-26,larvalabs,0.05,855,None,None,[Female],"[Bandana, Eye Patch]"
3710,Transfer,0x2cbee4,Myph-Spa…,2021-03-06,2021-03-06,larvalabs,NaN,410,None,None,[Male],"[Headband, Eye Patch]"
82908,Offered,,,2021-07-14,2021-07-14,larvalabs,26.45,5348,None,None,[Female],"[Black Lipstick, Clown Hair Green]"
14887,Bid Withdrawn,0xa34d2d,,2021-08-09,2021-08-09,larvalabs,45.00,1369,None,None,[Female],"[Knitted Cap, Clown Eyes Blue]"
137917,Transfer,0xfc295a,0x758353,2021-01-18,2021-01-18,larvalabs,NaN,8415,None,None,[Male],"[Stringy Hair, Clown Eyes Green]"
4234,Offered,,,2021-05-09,2021-05-09,larvalabs,40.00,457,None,None,[Male],"[Chinstrap, Knitted Cap, 3D Glasses]"
135970,Offered,,,2019-12-30,2019-12-30,larvalabs,1.25,8313,None,None,[Male],"[Cigarette, Muttonchops, Top Hat]"
60350,Offered,,,2021-09-13,2021-09-13,larvalabs,85.00,3969,None,None,[Female],"[Mohawk Thin, Hot Lipstick, Cigarette]"
96723,Offer Withdrawn,,,2021-08-31,2021-08-31,larvalabs,NaN,6166,None,None,[Male],"[Eye Mask, Normal Beard Black, Hoodie]"


In [10]:
data = [original_df["punk_id"], original_df["type"]]
headers = ["punk_id", "type"]
basic_info = pd.concat(data, axis=1, keys=headers)
basic_info.sample(10)

,punk_id,type
55797,3719,[Male]
163980,9819,[Male]
138283,8435,[Male]
127324,7804,[Alien]
155742,9374,[Male]
79575,5158,[Female]
37244,2738,[Male]
51285,3492,[Female]
109427,6810,[Female]
87765,5644,[Female]


In [13]:
basic_info.count

<bound method DataFrame.count of         punk_id      type
0             0  [Female]
1             0  [Female]
2             0  [Female]
3             0  [Female]
4             0  [Female]
...         ...       ...
167487     9999  [Female]
167488     9999  [Female]
167489     9999  [Female]
167490     9999  [Female]
167491     9999  [Female]

[167492 rows x 2 columns]>

In [14]:
basic_info.sort_values("punk_id", inplace = True)

In [17]:
basic_info.drop_duplicates(subset ="punk_id",
                     keep = False, inplace = True)
basic_info.sample(20)

,punk_id,type
33820,2531,[Female]
154346,9302,[Male]
136029,8318,[Male]
3296,371,[Female]
3065,343,[Male]
5521,582,[Male]
71577,4655,[Male]
157684,9469,[Male]
115113,7125,[Female]
23764,1918,[Female]


In [20]:
basic_info.reset_index(drop=True, inplace=True)
basic_info.count

<bound method DataFrame.count of      punk_id      type
0         11  [Female]
1         18    [Male]
2         26  [Female]
3         27    [Male]
4         29  [Female]
..       ...       ...
632     9831  [Female]
633     9894    [Male]
634     9930    [Male]
635     9936    [Male]
636     9940  [Female]

[637 rows x 2 columns]>

In [21]:
basic_info.head(50)

,punk_id,type
0,11,[Female]
1,18,[Male]
2,26,[Female]
3,27,[Male]
4,29,[Female]
5,31,[Female]
6,34,[Female]
7,37,[Female]
8,38,[Female]
9,41,[Male]
